<a href="https://colab.research.google.com/github/lukasagurkas/HealthRaceApp/blob/master/OLS_LinReg_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Reading the data

In [1]:
import pandas as pd
import numpy as np

url_transactions = 'https://drive.google.com/file/d/11HedlzHhHPcwQ9U8CHNVUKlB9mrEwiVK/view?usp=sharing'
path_transactions = 'https://drive.google.com/uc?export=download&id='+url_transactions.split('/')[-2]
df_transactions = pd.read_csv(path_transactions)

url_promotions = 'https://drive.google.com/file/d/1A-vU0zzlcuVnDKyvHLqJjPGg72hM709O/view?usp=sharing'
path_promotions = 'https://drive.google.com/uc?export=download&id='+url_promotions.split('/')[-2]
df_promotions = pd.read_csv(path_promotions)

url_products = 'https://drive.google.com/file/d/1KVGV3EoB3Z44-byGBcoyX6X6e9SoPjCH/view?usp=sharing'
path_products = 'https://drive.google.com/uc?export=download&id='+url_products.split('/')[-2]
df_products = pd.read_csv(path_products)

url_inventory = 'https://drive.google.com/file/d/1ia8BZQfOp7e3uJm9F4WL7BbIJAoJDSQn/view?usp=sharing'
path_inventory = 'https://drive.google.com/uc?export=download&id='+url_inventory.split('/')[-2]
df_inventory = pd.read_csv(path_inventory)

## Transactions

In [ ]:
df_transactions

,day,time,customer,bank acount,category,product_id,description,size,std_sales_price,purchase_price,bio,basic
0,1/1/2018,9:07:00,0.0,NaN,snack,s_5,Eierkoeken,4st,1.79,1.79,0.0,0.0
1,1/1/2018,9:07:00,0.0,NaN,meat,m_16,Sappige gelders rookworst,250g,1.08,1.08,0.0,0.0
2,1/1/2018,9:07:00,0.0,NaN,meat,m_18,Varkensbraadworst,4st,2.49,1.99,NaN,0.0
3,1/1/2018,9:07:00,0.0,NaN,meat,m_6,Half om half gehakt,500g,2.69,2.42,0.0,0.0
4,1/1/2018,9:07:00,0.0,NaN,bread,b_33,Biologisch bruin heel,1st,2.12,2.12,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
354860,30-12-2018,20:59:33,20685.0,NaN,meat,m_1,Rundergehakt,500g,3.69,3.69,0.0,0.0
354861,30-12-2018,20:59:33,20685.0,NaN,fruit,f_26,Biologische rozijnen,250g,2.99,2.99,1.0,0.0
354862,30-12-2018,20:59:33,20685.0,NaN,meat,m_24,Gekruide runderchipolata,8st,3.19,3.19,0.0,0.0
354863,30-12-2018,20:59:33,20685.0,NaN,meat,m_18,Varkensbraadworst,4st,2.49,2.49,0.0,0.0


In [ ]:
df_transactions.describe()

### Data preprocessing

In [2]:
#Droping rows with Nan value in columns day, time, purchase_price, customer and product_id

df_transactions.dropna(subset= ['day', 'time', 'purchase_price','customer', 'product_id'],inplace = True)

### Linear Regression (dependent variable - quanitity purchased per week per product; independent variable - price of all products per week)

In [4]:
df_nrOfPromotionsPerProduct = df_promotions.groupby(by=[('product_id')]).count()
df_nrOfPromotionsPerProduct

,week,category,description,discount
product_id,,,,
b_1,5,5,5,5
b_10,5,5,5,5
b_11,6,6,6,6
b_12,7,7,7,7
b_13,7,7,7,7
...,...,...,...,...
v_5,5,5,5,5
v_6,8,8,8,8
v_7,4,4,4,4


In [5]:
df_nrOfPromotionsPerProduct.describe()

,week,category,description,discount
count,152.000000,152.000000,152.000000,152.000000
mean,5.131579,5.131579,5.131579,5.131579
std,2.067357,2.067357,2.067357,2.067357
min,1.000000,1.000000,1.000000,1.000000
25%,4.000000,4.000000,4.000000,4.000000
50%,5.000000,5.000000,5.000000,5.000000
75%,6.000000,6.000000,6.000000,6.000000
max,14.000000,14.000000,14.000000,14.000000


In [6]:
df_nrOfPromotionsPerProduct = df_nrOfPromotionsPerProduct.reset_index()
df_nrOfPromotionsPerProduct

,product_id,week,category,description,discount
0,b_1,5,5,5,5
1,b_10,5,5,5,5
2,b_11,6,6,6,6
3,b_12,7,7,7,7
4,b_13,7,7,7,7
...,...,...,...,...,...
147,v_5,5,5,5,5
148,v_6,8,8,8,8
149,v_7,4,4,4,4
150,v_8,7,7,7,7


In [7]:
# We want to remove the products with a low number of discounts because they
# negatively affect the OLS data
productsWithGoodAmountOfDiscounts = []

for index, row in df_nrOfPromotionsPerProduct.iterrows():
  if row['week'] >= 5:
    productsWithGoodAmountOfDiscounts.append(row['product_id'])

In [8]:
len(productsWithGoodAmountOfDiscounts)

93

In [9]:
for index, row in df_products.iterrows():
  if row['product_id'] not in productsWithGoodAmountOfDiscounts:
    df_products.drop([index], inplace=True)

df_products

,category,product_id,description,size,std_sales_price,bio,basic
1,vegetable,v_2,Courgette,1st,0.75,0,0
3,vegetable,v_4,Paprika Mix,3st,1.79,0,0
4,vegetable,v_5,Paprika,1st,0.99,0,0
5,vegetable,v_6,Geschrapte worteltjes,500g,1.59,0,0
7,vegetable,v_8,Basis Rode kool met appel,660g,0.65,0,1
...,...,...,...,...,...,...,...
150,snack,s_4,Bakkersspeculaas,1st,0.81,0,1
151,snack,s_5,Eierkoeken,4st,1.79,0,0
152,snack,s_6,Ribbelchips naturel,250g,0.99,0,1
153,snack,s_7,Ribbelchips paprika,250g,0.99,0,1


In [10]:
for index, row in df_transactions.iterrows():
  if row['product_id'] not in productsWithGoodAmountOfDiscounts:
    df_transactions.drop([index], inplace=True)

df_transactions

,day,time,customer,bank acount,category,product_id,description,size,std_sales_price,purchase_price,bio,basic
0,1/1/2018,9:07:00,0.0,NaN,snack,s_5,Eierkoeken,4st,1.79,1.79,0.0,0.0
2,1/1/2018,9:07:00,0.0,NaN,meat,m_18,Varkensbraadworst,4st,2.49,1.99,NaN,0.0
3,1/1/2018,9:07:00,0.0,NaN,meat,m_6,Half om half gehakt,500g,2.69,2.42,0.0,0.0
6,1/1/2018,9:07:00,0.0,NaN,snack,s_1,Vanille muffins (vers),1st,3.07,3.07,0.0,0.0
9,1/1/2018,9:07:00,0.0,NaN,fruit,f_7,Pitloze rode druiven,500g,2.19,2.19,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
354860,30-12-2018,20:59:33,20685.0,NaN,meat,m_1,Rundergehakt,500g,3.69,3.69,0.0,0.0
354861,30-12-2018,20:59:33,20685.0,NaN,fruit,f_26,Biologische rozijnen,250g,2.99,2.99,1.0,0.0
354862,30-12-2018,20:59:33,20685.0,NaN,meat,m_24,Gekruide runderchipolata,8st,3.19,3.19,0.0,0.0
354863,30-12-2018,20:59:33,20685.0,NaN,meat,m_18,Varkensbraadworst,4st,2.49,2.49,0.0,0.0


In [11]:
df_transactions.to_csv('transactionsWithoutProductsWithFewDiscounts.csv')

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Initializing result array
d = {'product_id': df_products['product_id']} # Dictionary with all products
df_linreg_results = pd.DataFrame(data=d) # DataFrame where results will be stored

# Iterating through each product
for index, row_1 in df_products.iterrows():

  # Quanitity of product (with product_id == row_1['product_id']) purchased per 
  # week (the duration of promotions) (y), the dependent variable
  df_temp_1 = df_transactions[df_transactions['product_id'] == row_1['product_id']].copy()
  df_temp_1['day'] = pd.to_datetime(df_temp_1['day'])
  df_temp_2 = df_temp_1.resample('W', on = 'day').product_id.count()
  df_temp_2 = df_temp_2.to_frame()
  y = np.array(np.log(df_temp_2['product_id']))

  # Initialization of array of independent variables (x) with the first product
  # ('v_1'), the independent variables are purchase prices of each product per 
  # week (the duration of promotions)
  df_temp_3 = df_transactions[df_transactions['product_id'] == 'v_2'].copy()
  df_temp_3['day'] = pd.to_datetime(df_temp_3['day'])
  df_temp_4 = df_temp_3.resample('W', on = 'day').purchase_price.median()
  df_temp_4 = df_temp_4.to_frame()
  x = np.array(np.log(df_temp_4['purchase_price']))

  # Putting the rest of the independent variables in array x
  for index, row_2 in df_products.iterrows():
    if (row_2['product_id'] != 'v_2'):
      df_temp_5 = df_transactions[df_transactions['product_id'] == row_2['product_id']].copy()
      df_temp_5['day'] = pd.to_datetime(df_temp_5['day'])
      df_temp_6 = df_temp_5.resample('W', on = 'day').purchase_price.median()
      df_temp_6 = df_temp_6.to_frame()
      x1 = np.array(np.log(df_temp_6['purchase_price']))
      x = np.column_stack((x, x1))

  #x_train, x_test, y_train, y_test = train_test_split(x, y)

  linreg_model = LinearRegression(normalize=True).fit(x, y)
  linreg_prediction = linreg_model.predict(x)
  #linreg_mae = np.mean(np.abs(y_test - linreg_prediction))

  # Storing the Linear Regression model's coefficients for every product
  df_linreg_results[row_1['product_id']] = linreg_model.coef_

df_linreg_results

,product_id,v_2,v_4,v_5,v_6,v_8,v_9,v_13,v_14,v_15,v_18,v_19,v_22,v_26,v_27,v_29,v_30,v_31,v_32,v_33,v_35,f_3,f_4,f_5,f_6,f_7,f_10,f_13,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_23,f_24,f_25,f_26,f_27,...,m_21,m_24,m_26,m_27,m_28,m_29,m_30,m_31,m_32,m_33,m_37,m_38,b_1,b_2,b_4,b_6,b_10,b_11,b_12,b_13,b_17,b_19,b_24,b_25,b_26,b_28,b_29,b_30,b_31,b_32,b_35,b_38,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8
1,v_2,-1.721601,3.686607,-0.917178,-0.214785,-0.097108,1.673669,1.323104,0.143770,-1.509195,-0.475058,0.773458,1.237327,0.390910,1.929292,-0.162982,1.683168,1.698877,2.813072,2.255669,-1.090735,1.331779,2.023419,0.740564,1.667375,-0.221488,1.015051,3.030415,-1.136955,1.184133,-0.368326,-1.401858,0.050850,0.065706,-0.073268,0.940781,-0.111373,-0.608069,1.163275,-0.052870,...,-1.204714,1.319156,-1.028142,0.407360,1.742585,-0.629325,2.094124,0.753139,-2.643468,2.173217,0.156959,-1.167095,-0.895736,2.191094,0.458328,-1.214943,2.709693,0.262844,0.357251,-1.314920,2.164148,-0.385825,2.851222,0.245255,0.894600,-1.074539,-0.017530,-1.189328,0.154998,0.023082,3.768430,2.318249,0.888342,-1.119806,0.709789,-2.133199,-0.289722,-0.621712,1.091527,0.916494
3,v_4,-0.029465,-0.836390,-1.041441,-0.102643,-0.637707,0.031716,0.523479,0.012225,-0.404887,-0.146262,0.461507,-0.013654,0.151501,-0.693161,-0.420686,0.093175,0.398914,-0.082609,-0.319926,-0.531383,-0.425407,-0.323364,-0.654283,0.117110,-0.675205,0.092092,0.252878,-1.283600,-0.159289,-0.463013,-0.441796,-1.346497,-0.033425,-0.721139,0.315540,-0.581849,-0.226738,-0.755789,-0.445184,...,-0.697013,-0.171625,-0.506229,0.150541,0.215326,-1.204123,0.145304,-0.957400,-0.863614,0.846004,1.509536,-0.477066,-0.360841,0.240515,0.485674,-1.262678,-0.184070,-0.966880,-0.588985,-0.411034,0.261980,-0.364092,0.171660,0.356659,-0.112451,-0.675258,-0.739518,-1.280078,0.359379,0.062460,0.498918,0.120179,-0.236306,-0.231529,-0.076878,-0.987414,-0.475029,-0.968862,-0.128142,-0.277907
4,v_5,0.119860,-0.188876,-0.132002,0.239646,0.535453,-0.202459,-0.139633,0.397112,-0.204608,-0.065941,-0.377831,0.182848,0.814212,-0.260926,0.045873,0.035585,0.315739,0.068590,-0.338276,0.513391,-0.606932,-0.189236,-0.263907,-0.156147,-0.488872,-0.257359,-0.746550,0.088409,-0.213627,0.088331,0.161488,-0.333198,0.936480,0.106221,-0.571923,0.302180,-0.192247,0.017304,-0.166344,...,1.297246,0.210727,-0.138838,-0.134515,0.061644,-0.061827,-0.401621,0.475473,0.189469,0.488626,0.484171,0.431178,0.018134,-0.624098,-0.298095,0.228584,-0.816295,0.085624,-0.525706,0.519701,-0.045558,-0.054373,-0.168161,0.074192,-0.024486,0.709469,0.231185,1.063763,0.350392,0.026024,-0.382303,-0.301979,-0.075809,0.224261,0.101120,0.517762,0.382643,0.779265,-0.451322,0.405758
5,v_6,0.392678,0.883562,-0.878005,-0.987973,0.396398,0.020422,0.412779,0.270691,0.346807,0.101106,0.624254,0.313283,0.049619,0.396496,-0.264612,0.520886,0.394479,-0.053102,0.525127,-0.064406,0.391943,0.439591,0.688497,1.575342,0.087435,0.077327,1.082995,-0.470820,0.175029,0.073025,-1.036195,0.234797,0.527980,-0.018708,1.122361,-0.207351,-0.274212,-0.198190,-0.427302,...,-0.505961,0.402916,-0.321320,-0.293613,0.209239,-1.163080,0.934800,-0.038239,0.264320,1.226776,0.890754,0.482735,-0.066196,-0.130672,-0.579811,-0.935817,-0.024180,-0.193800,-1.076128,-0.266065,0.040420,-0.722690,0.368652,-0.283893,-0.303574,-0.205412,-0.228880,-0.349203,0.796013,0.057882,0.757941,0.330259,-0.003803,-0.319926,0.014876,-0.372998,-0.250531,-0.427171,0.452241,-0.138249
7,v_8,0.045753,0.012109,-0.059242,0.393943,0.178235,-0.219749,-0.273445,-0.325416,-0.076143,-0.244556,0.057975,-0.069064,0.091167,-0.014934,-0.012318,-0.106240,-0.155961,0.259971,-0.206673,0.000626,-0.440283,-0.061990,0.205028,-0.174940,0.018442,-0.576376,-0.390207,0.263401,-0.241742,0.350916,-0.508601,0.150567,-0.008519,-0.217859,-0.133943,-0.119541,-0.155145,0.187638,-0.233150,...,0.117352,-0.209698,-0.407293,-0.387612,0.002434,-0.216207,-0.064356,0.041430,-0.604689,-0.109985,0.406637,-0.412360,-0.238125,-0.183314,-0.215595,0.714830,-0.085187,0.111082,-0.14946

In [18]:
df_linreg_results.to_csv('linreg_all_products_cleaned.csv')

In [19]:
df_linreg_results.reindex(df_linreg_results.v_2.abs().sort_values().index)

,product_id,v_2,v_4,v_5,v_6,v_8,v_9,v_13,v_14,v_15,v_18,v_19,v_22,v_26,v_27,v_29,v_30,v_31,v_32,v_33,v_35,f_3,f_4,f_5,f_6,f_7,f_10,f_13,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_23,f_24,f_25,f_26,f_27,...,m_21,m_24,m_26,m_27,m_28,m_29,m_30,m_31,m_32,m_33,m_37,m_38,b_1,b_2,b_4,b_6,b_10,b_11,b_12,b_13,b_17,b_19,b_24,b_25,b_26,b_28,b_29,b_30,b_31,b_32,b_35,b_38,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8
94,m_30,0.000118,-0.164467,0.059289,0.220689,0.236799,0.107459,-0.587405,-0.170481,0.129376,-0.407184,-0.103925,0.066985,0.380467,-0.114092,-0.030238,-0.093618,0.055846,0.003891,0.102690,0.156661,0.188159,0.661257,-0.083473,0.131272,-0.209670,0.188531,-0.454948,0.068086,-0.111510,0.121138,-0.296031,0.243844,0.445794,0.451778,-0.569131,0.170578,-0.563807,-0.047106,-0.041562,...,0.748829,0.252127,-0.076310,0.293571,0.357438,0.482556,-0.656302,0.274203,0.149839,0.006358,0.310898,-0.535750,-0.126575,-0.202780,0.345543,0.046016,0.027826,0.074989,0.012012,0.209336,-0.066140,-0.117274,-0.509044,-0.041267,0.611023,0.214433,0.330571,0.683871,-0.230637,0.039674,0.112161,0.278961,-0.199333,-0.040344,-0.038153,0.250511,0.521731,0.679046,-0.060607,0.716750
130,b_25,0.000944,0.833728,-0.495373,-0.377267,-0.023697,0.626717,-0.671306,-0.094353,-0.406229,-0.841579,-0.510279,-0.526013,-0.454206,0.027966,0.028192,0.187750,-0.021463,0.162976,0.220188,-0.298391,0.164873,0.184981,-0.246982,-0.234853,-0.811769,-0.050945,0.175115,-0.072648,-0.580665,-0.766542,-0.400850,0.120511,0.145393,0.302357,-0.013154,-0.008889,-0.795602,-0.073216,-0.317008,...,0.121468,0.038722,-0.204295,-0.523121,-0.064355,-0.396465,-0.085076,0.945604,-0.897490,0.084011,-1.252801,-0.395508,-0.492682,0.520003,-0.156136,-0.251862,0.104102,-0.095482,0.134314,-0.391537,0.855105,-0.376401,0.347946,-0.286754,0.093676,-0.117732,0.227328,0.890221,0.032515,0.027521,0.231987,0.058886,-0.068249,0.246290,-0.164083,0.301103,0.196226,0.158096,0.068451,0.138500
137,b_32,0.005685,0.025680,0.209185,0.498404,0.528768,0.187687,0.229257,0.958193,0.186401,0.077002,-0.643447,-0.296158,0.020259,0.228595,0.204954,-0.184584,0.160082,0.138246,-0.242910,0.028549,0.095188,0.340480,0.255231,0.117890,0.185768,-0.296421,-0.329101,1.076868,-0.259484,0.104395,-0.197901,0.478195,0.097300,0.636636,-0.484489,0.256179,-0.373650,0.229548,0.155607,...,0.657681,0.306663,0.304281,0.183924,-0.446039,0.571521,-0.023522,0.117084,0.062094,-0.088912,-0.415083,0.015073,0.272209,-0.598030,-0.127678,0.220586,-0.034607,-0.093977,-0.242599,0.248086,0.006823,-0.162849,-0.572242,-0.024888,0.137958,0.217304,0.156091,0.905614,0.214066,-0.170262,-0.236753,0.061911,0.029486,0.259711,-0.094172,0.317673,0.257225,0.863129,-0.028537,0.581113
25,v_26,0.006720,0.425373,-0.179618,0.342014,0.779381,0.926793,1.624009,0.735506,-1.029166,-0.946659,-0.295172,0.303099,-0.914674,-0.647559,-0.781186,-0.018201,1.302125,0.518225,-0.233226,-0.068772,0.163184,-0.160715,-0.348698,1.569419,0.525259,0.290007,2.062937,1.201197,-0.231682,-0.224066,-0.659356,-0.420916,0.264573,-0.232074,0.363864,-0.131735,-0.145151,-0.029147,-0.100048,...,-0.058935,0.756559,-0.748299,-0.086747,-0.762602,-0.866048,0.640396,-1.139109,-2.499437,0.825777,-0.135697,-0.845784,-0.333228,0.008312,-0.642958,-1.090265,0.418252,0.132895,-0.944275,-0.853463,0.127912,0.041081,0.560238,1.359655,0.873759,-0.366591,-0.064988,-0.270224,1.024540,0.078117,0.672453,0.588123,-0.127856,0.606250,0.014419,-0.454621,0.349353,-0.094109,0.661183,0.020519
84,m_20,0.013317,0.778058,0.133142,-0.332867,0.756847,0.449043,1.033013,0.217285,0.945437,0.139008,-0.697457,-0.141681,-0.096305,0.287163,0.331107,-0.135247,0.330398,0.640380,0.241624,0.016977,0.681640,0.071690,0.034962,1.215165,-0.039478,1.151832,-0.009283,0.949677,0.104792,0.045030,-0.283644,0.040980,-0.230469,0.459682,-0.602986,0.392658,-0.435613,0.588431,0.506930,...,1.124426,0.988103,-0.464073,-0.173021,-0.302589,0.185423,-0.037696,-0.286330,-0.373861,1.030612,0.139654,0.532287,-0.027184,-1.000180,-0.422673,0.353519,0.043205,1.379438,-0.091606,-0.368584,0.160288,0.5

In [20]:
df_temp = pd.read_csv('linreg_all_products_cleaned.csv')  

In [ ]:
df_temp.reindex(df_temp.v_1.abs().sort_values().index)

#### Top 5 positive coefficients

In [21]:
df_temp_describe = df_temp.describe()
max_values_0 = df_temp_describe.iloc[7].to_numpy()
max_values = np.sort(max_values_0)
index_min = max_values.size-6
index_max = max_values.size-1
max_values = max_values[index_min: index_max]

df_temp_describe_max = df_temp_describe.iloc[7]
columns_max_values = []
for a in max_values:
  for i in df_temp_describe.iloc[7].index:
    if df_temp_describe_max[i] == a:
      columns_max_values.append(i)

df_temp_1 = df_temp.set_index('product_id')

rows_max_values = []

for a in range(max_values.size):
  for index, row in df_temp_1.iterrows():
    if (row[columns_max_values[a]] == max_values[a]):
      rows_max_values.append(index)

max_values, columns_max_values, rows_max_values

(array([ 4.20051903,  4.85146453,  5.63817394,  7.12512519, 10.81267263]),
 ['b_2', 'm_8', 'm_32', 'm_17', 'm_37'],
 ['m_29', 'm_29', 'b_38', 'b_38', 'b_19'])

#### Top 5 negative coefficients

In [22]:
df_temp_describe = df_temp.describe()
min_values_0 = df_temp_describe.iloc[3].to_numpy()
min_values = np.sort(min_values_0)
index_min = 0
index_max = 5
min_values = min_values[index_min: index_max]

df_temp_describe_min = df_temp_describe.iloc[3]
columns_min_values = []
for a in min_values:
  for i in df_temp_describe.iloc[3].index:
    if df_temp_describe_min[i] == a:
      columns_min_values.append(i)

df_temp_1 = df_temp.set_index('product_id')

rows_min_values = []

for a in range(min_values.size):
  for index, row in df_temp_1.iterrows():
    if (row[columns_min_values[a]] == min_values[a]):
      rows_min_values.append(index)

min_values, columns_min_values, rows_min_values

(array([-7.69092465, -5.98266214, -5.43871078, -5.14189124, -5.08924056]),
 ['m_17', 'm_29', 'b_19', 'm_14', 'm_1'],
 ['m_29', 'm_29', 'm_29', 'm_29', 'm_29'])

#### Linear regression for top negative and positive relations of the Lasso model


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Most significant positive relations of the lasso model
columns_max_values_lasso = ['v_16', 'b_14', 'm_38', 'm_15', 'm_9']
rows_max_values_lasso = ['b_8', 'm_29', 'v_11', 'f_26', 'b_29']

# Most significant negative relations of the lasso model
columns_min_values_lasso = ['m_27', 's_4', 's_2', 'v_22', 'f_7']
rows_min_values_lasso = ['b_23', 'b_37', 'v_10', 'v_22', 'v_25']

row_index = rows_max_values_lasso + rows_min_values_lasso

column_index = columns_max_values_lasso + columns_min_values_lasso

# Initializing result array
d = {'product_id': row_index} # Dictionary with dependent variables which had
                      # the biggest and smallest coefficients in the Lasso model
df_linreg_results = pd.DataFrame(data=d) # DataFrame where results will be stored

# Iterating through each product in column_index
for i in column_index:

  # Quanitity of product (with product_id == row_1['product_id']) purchased per 
  # week (the duration of promotions) (y), the dependent variable
  df_temp_1 = df_transactions[df_transactions['product_id'] == i].copy()
  df_temp_1['day'] = pd.to_datetime(df_temp_1['day'])
  df_temp_2 = df_temp_1.resample('W', on = 'day').product_id.count()
  df_temp_2 = df_temp_2.to_frame()
  y = np.array(np.log(df_temp_2['product_id']))

  # Initialization of array of independent variables (x) with the first product
  # ('b_8'), the independent variables are purchase prices of each product per 
  # week (the duration of promotions)
  df_temp_3 = df_transactions[df_transactions['product_id'] == 'b_8'].copy()
  df_temp_3['day'] = pd.to_datetime(df_temp_3['day'])
  df_temp_4 = df_temp_3.resample('W', on = 'day').purchase_price.median()
  df_temp_4 = df_temp_4.to_frame()
  x = np.array(np.log(df_temp_4['purchase_price']))

  # Putting the rest of the independent variables in array x
  for j in row_index:
    if (j != 'b_8'):
      df_temp_5 = df_transactions[df_transactions['product_id'] == j].copy()
      df_temp_5['day'] = pd.to_datetime(df_temp_5['day'])
      df_temp_6 = df_temp_5.resample('W', on = 'day').purchase_price.median()
      df_temp_6 = df_temp_6.to_frame()
      x1 = np.array(np.log(df_temp_6['purchase_price']))
      x = np.column_stack((x, x1))

  x_train, x_test, y_train, y_test = train_test_split(x, y)

  linreg_model = LinearRegression(normalize=True).fit(x_train, y_train)
  linreg_prediction = linreg_model.predict(x_test)
  linreg_mae = np.mean(np.abs(y_test - linreg_prediction))

  # Storing the Linear Regression model's coefficients for every product
  df_linreg_results[i] = linreg_model.coef_

df_linreg_results

,product_id,v_16,b_14,m_38,m_15,m_9,m_27,s_4,s_2,v_22,f_7
0,b_8,0.295924,0.945963,0.163383,1.673972,0.329140,0.517348,-0.129360,-0.224715,0.082377,0.223449
1,m_29,2.522670,2.433598,1.214602,-1.077293,0.562491,-1.045574,0.119213,-0.063090,1.903698,-0.930058
2,v_11,-3.144759,-3.237714,-1.698668,6.634781,1.865427,-4.513714,-1.031824,-1.268280,3.253747,3.236080
3,f_26,0.462603,-0.165374,0.394819,5.069223,-0.611549,0.541710,1.037417,0.126484,-0.328125,1.146994
4,b_29,-0.293697,-0.077221,-0.306382,-0.286478,-1.396757,-0.670309,8.711184,0.557760,0.602972,-0.303854
5,b_23,-0.647004,-0.627632,-0.071298,-7.566499,0.891515,-11.964966,-1.366241,-3.732777,-4.050081,3.458135
6,b_37,22.643243,0.831179,-4.333776,9.810475,-2.393268,-5.720392,-0.692754,-3.611640,1.193350,1.639046
7,v_10,1.741536,0.578246,0.490907,1.487244,1.256946,0.817079,-0.324738,-0.123430,-0.435380,1.770376
8,v_22,0.541425,-0.327002,0.242446,-5.077948,2.263114,0.070898,-1.370013,0.690880,-2.197378,-0.346618
9,v_25,-0.829024,0.185453,0.550703,1.771180,-0.410751,-3.540549,0.732237,-0.098139,0.178264,-1.221560


In [ ]:
df_linreg_results.to_csv('linreg_top_lasso_products.csv')

In [ ]:
df_linreg_results.reindex(df_linreg_results.v_2.abs().sort_values().index)

### Linear Regression method (dependent variable - quanitity purchased per week per product; independent variable - price of products per category per week) 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# foreach product in category category
for i in df_products.category.unique():

  # Initializing result array
  df_products_category_products = df_products[df_products['category'] == i]
  # Dictionary within the current category
  d = {'product_id': df_products_category_products['product_id']}
  df_linreg_results = pd.DataFrame(data=d) # DataFrame where results will be stored
  df_linreg_results = df_linreg_results.reset_index()
  df_linreg_results = df_linreg_results[['product_id']]

  # Iterating through each product in the category
  for index, row_1 in df_linreg_results.iterrows():

    # Quanitity of product (with product_id == row_1['product_id']) purchased per 
    # week (the duration of promotions) (y), the dependent variable
    df_temp_1 = df_transactions[df_transactions['product_id'] == row_1['product_id']].copy()
    df_temp_1['day'] = pd.to_datetime(df_temp_1['day'])
    df_temp_2 = df_temp_1.resample('W', on = 'day').product_id.count()
    df_temp_2 = df_temp_2.to_frame()
    y = np.array(np.log(df_temp_2['product_id']))

    # Initialization of array of independent variables (x) with the product being
    # investeigated, the independent variables are purchase prices of each product per 
    # week (the duration of promotions)
    df_temp_3 = df_transactions[df_transactions['product_id'] == row_1['product_id']].copy()
    df_temp_3['day'] = pd.to_datetime(df_temp_3['day'])
    df_temp_4 = df_temp_3.resample('W', on = 'day').purchase_price.median()
    df_temp_4 = df_temp_4.to_frame()
    x = np.array(np.log(df_temp_4['purchase_price']))

    # Putting the rest of the independent variables in array x
    for index, row_2 in df_linreg_results.iterrows():
      if (row_2['product_id'] != row_1['product_id']):
        df_temp_5 = df_transactions[df_transactions['product_id'] == row_2['product_id']].copy()
        df_temp_5['day'] = pd.to_datetime(df_temp_5['day'])
        df_temp_6 = df_temp_5.resample('W', on = 'day').purchase_price.median()
        df_temp_6 = df_temp_6.to_frame()
        x1 = np.array(np.log(df_temp_6['purchase_price']))
        x = np.column_stack((x, x1))

    x_train, x_test, y_train, y_test = train_test_split(x, y)

    linreg_model = LinearRegression(normalize=True).fit(x_train, y_train)
    linreg_prediction = linreg_model.predict(x_test)
    linreg_mae = np.mean(np.abs(y_test - linreg_prediction))

    # Storing the Linear Regression model's coefficients for every product
    df_linreg_results[row_1['product_id']] = linreg_model.coef_

    if i == 'vegetable':
      df_vegetable_linreg = df_linreg_results
      df_vegetable_linreg.to_csv('vegetable_linreg.csv')
    elif i == 'fruit':
      df_fruits_linreg = df_linreg_results
      df_fruits_linreg.to_csv('fruits_linreg.csv')
    elif i == 'meat':
      df_meat_linreg = df_linreg_results
      df_meat_linreg.to_csv('meat_linreg.csv')
    elif i == 'bread':
      df_bread_linreg = df_linreg_results
      df_bread_linreg.to_csv('bread_linreg.csv')
    elif i == 'snack':
      df_snack_linreg = df_linreg_results
      df_snack_linreg.to_csv('snack_linreg.csv')

df_vegetable_linreg

,product_id,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14,v_15,v_16,v_17,v_18,v_19,v_20,v_21,v_22,v_23,v_24,v_25,v_26,v_27,v_28,v_29,v_30,v_31,v_32,v_33,v_34,v_35
0,v_1,-1.113587,-1.100610,1.950951,-6.511910e+12,0.323031,-0.255452,-43.938591,-2.055808,-2.462232e+00,-8.090359e+14,-7.362539,1.873214e+13,-5.074077,-3.370877e+13,1.726498,-11328.101305,7.966477,-1.601547,-0.396438,-1.226813,8.995963,-1.977069,0.686561,50.681394,-67.437490,-2.430387e+13,140.669984,-1596.330574,0.114816,3.775857,-1.502994,-1.205000,-3.198191,6.621586e+15,-0.796502
1,v_2,1.669536,1.138704,-1.542027,4.090416e-01,-1.096090,0.757210,-0.831089,0.368356,1.024225e+00,6.378182e-01,-0.661515,-6.693409e-01,-4.296023,-1.205337e+00,-2.716780,-0.215001,-5.016191,-0.141841,1.876525,0.674400,58.361872,-0.887102,-0.392046,99.345236,-1.195793,2.635670e-01,5.182161,-0.542496,-6.904056,-0.670675,-0.446327,1.013295,-0.304442,-9.514127e-01,-0.050688
2,v_3,-2.759244,-0.734837,1.977814,2.235128e+13,2.784752,-0.010823,-11.725148,2.626734,2.195127e+00,1.224181e+00,1.541887,-6.306640e-01,4.350685,7.999552e+12,0.218892,35.428721,5.476821,1.262009,3.300687,-1.554655,2.243606,1.763763,1.733868,1.395279,1.246870,4.198686e+00,0.692952,20.401619,1.992664,-0.848813,1.418836,1.652924,-5.335363,6.108985e+00,-0.165857
3,v_4,16.257255,2.299906,19.057627,-2.450689e+12,0.284030,-1.858357,-8.009271,3.029209,1.897161e+00,-3.702626e+13,2.373990,-8.623530e+11,-1.970606,7.085908e+12,7.659320,-8.234149,-0.879536,124.083146,-3.361656,1.174960,-1.655871,1.466001,1.958692,-0.534929,-0.771811,3.713412e+12,-35.381561,0.292205,1.136071,-0.509246,2.463854,1.107271,1.312383,5.403589e+12,42.478348
4,v_5,-33.473043,3.342895,-3.023269,2.437698e+12,-0.610456,-2.030913,2.130409,4.021864,7.337021e+00,-7.243829e+13,4.347765,-4.142558e+13,-0.000896,-1.463322e+00,0.361188,-28.592101,-1.078293,905.337448,-2.252473,5.596277,-19.511011,-1.809894,-6.250668,-1.067493,-3.589108,2.428343e+13,-227.655336,-7.800201,7.834995,-0.285198,-23.421611,0.666959,0.896688,-7.872192e+11,291.438847
5,v_6,1.193291,1.181554,1.130497,-6.683735e-01,-0.970094,0.140605,13.242962,1.593128,-1.891499e-01,-8.727686e+12,1.211058,8.577817e+11,-4.770261,-7.048347e+12,-17.990453,0.095743,-0.407131,-119.071802,9.589842,-5.152011,10.863998,4.288435,15.136513,1.651843,1.363526,-2.692013e+12,110.240169,7.480297,-2.342580,7.733860,34.168839,-2.154709,2.340448,-5.374945e+12,-38.050269
6,v_7,0.677772,-0.081312,10.245596,-1.877056e+01,5.211025,-3.524410,0.322868,0.459182,1.354762e+00,2.248215e-01,1.033159,3.313464e-02,-0.929915,8.644874e-01,-0.990851,-3.248289,0.076292,1.274426,2.827951,1.118631,-1.475886,-0.265609,1.709450,-0.094315,0.588865,-6.433103e-01,1.486793,1.136563,1.624154,-0.469921,-0.395888,0.245823,1.120783,6.649674e-01,1.197425
7,v_8,-0.055195,0.538258,-2.792449,6.150837e-01,2.504020,2.158297,0.139303,8.500644,1.429438e+01,4.207163e-01,12.529499,9.622628e+00,-20.056881,-7.663877e+13,-1.531468,-67.019602,-8.449302,-46.112182,-5.316904,2.435760,-25.136231,8.337533,8.465308,0.395753,-1.826266,3.987393e+00,18.765217,-0.747424,6.124353,-12.774274,-26.270741,-16.794726,10.346454,-7.044354e+12,12.611435
8,v_9,-0.110704,-0.697281,-2.471785,1.599475e+00,1.738601,1.702968,-0.497748,-1.393395,-2.958969e+00,4.350931e-01,-0.749087,-3.087568e-01,3.512774,4.798653e+00,-1.704180,15.440733,1.807707,-0.573981,-4.857935,-0.349006,-2.429535,-1.163852,0.042551,-6.333458,-1.119786,-6.325891e-01,-5.340381,-0.938000,-8.749987,4.985254,0.692993,0.639043,-2.653460,-6.908583e-01,-0.429470
9,v_10,12.538782,-0.867127,14.392858,-2.239939e+13,-1.592974,1.384635,-7.121962,0.190241,-1.464457e+00,4.901601e-01,0.560026,-6.703755e-03,1.559800,8.265360e-01,0.101054,-2.550425,2.648357,2.951958,-0.434715,0.132461,-0.045820,0.329025,-1.227407,-0.960755,-2.425466,8.784808e-01,-0.915665,-0.387823,-0.261717,1.837518,-0.105412,-1.656501,0.590212,-2.186710e+00,-1.470521


In [ ]:
import pandas as pd
df_vegetable_linreg = pd.read_csv('vegetable_linreg.csv')
df_fruits_linreg = pd.read_csv('fruits_linreg.csv')
df_meat_linreg = pd.read_csv('meat_linreg.csv')
df_bread_linreg = pd.read_csv('bread_linreg.csv')
df_snack_linreg = pd.read_csv('snack_linreg.csv')

FileNotFoundError: ignored

## Promotions

In [ ]:
df_promotions.head()

,week,category,product_id,description,discount
0,1,vegetable,v_14,Biologische knoflook,30
1,1,vegetable,v_2,Courgette,5
2,1,vegetable,v_16,Biologische kikkererwten,10
3,1,vegetable,v_12,Biologische courgette,15
4,1,bread,b_41,Chinois brioche,30


In [ ]:
df_promotions.describe()

## Products

In [ ]:
df_products.head()

,category,product_id,description,size,std_sales_price,bio,basic
0,vegetable,v_1,Komkommer,1st,0.69,0,0
1,vegetable,v_2,Courgette,1st,0.75,0,0
2,vegetable,v_3,SnoepVegetable tomaat,500g,1.99,0,0
3,vegetable,v_4,Paprika Mix,3st,1.79,0,0
4,vegetable,v_5,Paprika,1st,0.99,0,0


In [ ]:
df_products.describe()

,std_sales_price,bio,basic
count,155.000000,155.000000,155.000000
mean,2.080452,0.154839,0.103226
std,1.107832,0.362923,0.305240
min,0.470000,0.000000,0.000000
25%,1.105000,0.000000,0.000000
50%,1.990000,0.000000,0.000000
75%,2.690000,0.000000,0.000000
max,5.890000,1.000000,1.000000


## Inventory

In [ ]:
df_inventory.head()

,day,before or after delivery,Komkommer,Courgette,SnoepVegetable tomaat,Paprika Mix,Paprika,Geschrapte worteltjes,Basis Sperziebonen,Basis Rode kool met appel,Basis worteltjes fijn,Basis erwten zeer fijn,Basis sperziebonen,Biologische courgette,Biologische Bloemkool,Biologische knoflook,Biologische pompoen,Biologische kikkererwten,Biologische prei,Biologische rode kool,Biologische cherry tomaten,Biologische brocolli,Boerenkool gesneden,Spruiten,Witlof,Rucola,Nasi bami Vegetable,Snijbonen,Buitenbeentjes puntpaprika,Basis Maiskorrels,Wortelen,Pompoen stukjes,Verse zuurkool,Buitenbeentjes komkommer,Buitenbeentjes paprika,SnoepVegetable worteltjes,WokVegetable thais,Mandarijenen,Bananen,Chiquita bananen,...,Tijger bruin half,Volkoren heel,Meergranen volkoren heel,Tijger wit half,Vloerbrood meergranen,Vloerbrood volkoren,Casino wit half,Tijger wit heel,Bruin heel,Zaanse sijnder bruin half,Boerenvolkoren sesam heel,Steiv spelt gesneden,Krenten- en rozijnenbrood half,Zaans volkoren,Atkins laag koolhydraat volkoren brood half,Bolletje Fries roggebrood,Kaneelbroodjes,Biologisch Vloerbrood meerzaden heel,Basic meergranen heel,Basic wit heel,Basic casino half,Basic volkoren heel,Biologisch volkoren heel,Biologisch bruin heel,Glutenvrij bruin,Muesli-notenbrood,Basic roggebrood,Melkbroodjes,Krenten rozijnenwegge,Amandelkrans,Roomboter chocolade brood,Chinois brioche,Vanille muffins (vers),Basic chocolate chip cookies,Oreo,Bakkersspeculaas,Eierkoeken,Ribbelchips naturel,Ribbelchips paprika,Doritos sweet chili
0,0,before,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
1,0,after,"[(14, 36)]","[(14, 8)]","[(14, 37)]","[(14, 33)]","[(14, 34)]","[(14, 35)]","[(14, 31)]","[(14, 35)]","[(14, 32)]","[(14, 34)]","[(14, 32)]","[(14, 16)]","[(14, 25)]","[(14, 30)]","[(14, 32)]","[(14, 10)]","[(14, 26)]","[(14, 26)]","[(14, 25)]","[(14, 26)]","[(14, 37)]","[(14, 35)]","[(14, 35)]","[(14, 35)]","[(14, 36)]","[(14, 48)]","[(14, 36)]","[(14, 7)]","[(14, 35)]","[(14, 37)]","[(14, 35)]","[(14, 35)]","[(14, 33)]","[(14, 37)]","[(14, 37)]","[(20, 29)]","[(20, 25)]","[(20, 24)]",...,"[(5, 20)]","[(5, 20)]","[(5, 20)]","[(5, 20)]","[(5, 20)]","[(5, 23)]","[(5, 22)]","[(5, 21)]","[(5, 20)]","[(5, 19)]","[(5, 22)]","[(5, 19)]","[(5, 21)]","[(5, 20)]","[(5, 21)]","[(5, 18)]","[(5, 19)]","[(5, 47)]","[(5, 36)]","[(5, 34)]","[(5, 35)]","[(5, 36)]","[(5, 46)]","[(5, 45)]","[(5, 13)]","[(5, 19)]","[(5, 36)]","[(5, 20)]","[(5, 20)]","[(5, 20)]","[(5, 19)]","[(5, 23)]","[(100, 130)]","[(100, 45)]","[(100, 131)]","[(100, 45)]","[(100, 125)]","[(100, 45)]","[(100, 44)]","[(100, 128)]"
2,3,before,"[(14, 13)]",[],"[(14, 13)]","[(14, 11)]",[],[],"[(14, 2)]","[(14, 12)]","[(14, 7)]","[(14, 14)]","[(14, 13)]","[(14, 3)]","[(14, 19)]","[(14, 17)]","[(14, 19)]",[],"[(14, 23)]","[(14, 19)]","[(14, 20)]","[(14, 19)]","[(14, 6)]","[(14, 11)]","[(14, 11)]","[(14, 17)]","[(14, 2)]",[],"[(14, 19)]",[],"[(14, 12)]","[(14, 18)]","[(14, 2)]","[(14, 9)]","[(14, 6)]","[(14, 9)]","[(14, 13)]",[],[],"[(20, 1)]",...,[],"[(5, 2)]","[(5, 1)]","[(5, 9)]","[(5, 4)]",[],"[(5, 7)]","[(5, 6)]","[(5, 9)]",[],"[(5, 13)]","[(5, 1)]","[(5, 2)]",[],"[(5, 13)]","[(5, 4)]","[(5, 3)]","[(5, 29)]","[(5, 12)]","[(5, 9)]","[(5, 2)]","[(5, 10)]","[(5, 23)]","[(5, 32)]",[],"[(5, 3)]","[(5, 17)]","[(5, 10)]","[(5, 4)]","[(5, 5)]",[],[],"[(100, 3)]",[],"[(100, 19)]","[(100, 7)]","[(100, 8)]","[(100, 10)]","[(100, 11)]","[(100, 26)]"
3,3,after,"[(14, 13), (17, 36)]","[(17, 8)]","[(14, 13), (17, 37)]","[(14, 11), (17, 33)]","[(17, 34)]","[(17, 35)]","[(14, 2), (17, 31)]","[(14, 12), (17, 35)]","[(14, 7), (17, 32)]","[(14, 14), (17, 34)]","[(14, 13), (17, 32)]","[(14, 3), (17, 16)]","[(14, 19), (17, 25)]","[(14, 17), (17, 30)]","[(14, 19), (17, 32)]","[(17, 10)]","[(14, 23), (17, 26)]","[(14, 19), (17, 26)]","[(14, 20), (17, 25)]"

In [ ]:
df_inventory.describe()

## Example database style join on columns

In [ ]:
df_merge_transaction_products = pd.merge(df_transactions, df_products, on=['product_id'])
df_merge_transaction_products.head()

,day,time,customer,bank acount,category_x,product_id,description_x,size_x,std_sales_price_x,purchase_price,bio_x,basic_x,category_y,description_y,size_y,std_sales_price_y,bio_y,basic_y
0,1/1/2018,9:07:00,0.0,NaN,snack,s_5,Eierkoeken,4st,1.79,1.79,0.0,0.0,snack,Eierkoeken,4st,1.79,0,0
1,1/1/2018,9:59:31,2.0,72461338.0,snack,s_5,Eierkoeken,4st,1.79,1.79,0.0,0.0,snack,Eierkoeken,4st,1.79,0,0
2,1/1/2018,10:16:33,3.0,25672381.0,snack,s_5,Eierkoeken,4st,1.79,1.79,0.0,0.0,snack,Eierkoeken,4st,1.79,0,0
3,1/1/2018,10:25:31,4.0,304428.0,snack,s_5,Eierkoeken,4st,1.79,1.79,0.0,0.0,snack,Eierkoeken,4st,1.79,0,0
4,1/1/2018,11:05:06,8.0,44183347.0,snack,s_5,Eierkoeken,4st,1.79,1.79,0.0,0.0,snack,Eierkoeken,4st,1.79,0,0
